# Read SpaceX Launch Data from https://api.spacexdata.com/v4

Documentation of the SpaceX API can be found @ https://github.com/r-spacex/SpaceX-API/tree/master/docs

In [29]:
import requests

# Pick a dataset and the appropriate url (will require url+"<data>" in other calls to pull specific data)
url = "https://api.spacexdata.com/v4/"
# Get launch data
launches = requests.get(url+"launches").json()
# Get payload data
payloads = requests.get(url+"payloads").json()
# Get rockets data
rockets = requests.get(url+"rockets").json()

In [30]:
print(f"There are {len(data)} launches")
print(f"There are {len(payloads)} payloads")
print(f"There are {len(rockets)} rockets")

There are 205 launches
There are 225 payloads
There are 4 rockets


In [31]:
for launch in data:
  print(launch.get('name'), launch.get('date_utc'))

FalconSat 2006-03-24T22:30:00.000Z
DemoSat 2007-03-21T01:10:00.000Z
Trailblazer 2008-08-03T03:34:00.000Z
RatSat 2008-09-28T23:15:00.000Z
RazakSat 2009-07-13T03:35:00.000Z
Falcon 9 Test Flight 2010-06-04T18:45:00.000Z
COTS 1 2010-12-08T15:43:00.000Z
COTS 2 2012-05-22T07:44:00.000Z
CRS-1 2012-10-08T00:35:00.000Z
CRS-2 2013-03-01T19:10:00.000Z
CASSIOPE 2013-09-29T16:00:00.000Z
SES-8 2013-12-03T22:41:00.000Z
Thaicom 6 2014-01-06T18:06:00.000Z
CRS-3 2014-04-18T19:25:00.000Z
OG-2 Mission 1 2014-07-14T15:15:00.000Z
AsiaSat 8 2014-08-05T08:00:00.000Z
AsiaSat 6 2014-09-07T05:00:00.000Z
CRS-4 2014-09-21T05:52:00.000Z
CRS-5 2015-01-10T09:47:00.000Z
DSCOVR 2015-02-11T23:03:00.000Z
ABS-3A / Eutelsat 115W B 2015-03-02T03:50:00.000Z
CRS-6 2015-04-14T20:10:00.000Z
TürkmenÄlem 52°E / MonacoSAT 2015-04-27T23:03:00.000Z
CRS-7 2015-06-28T14:21:00.000Z
OG-2 Mission 2 2015-12-22T01:29:00.000Z
Jason 3 2016-01-17T15:42:00.000Z
SES-9 2016-03-04T23:35:00.000Z
CRS-8 2016-04-08T20:43:00.000Z
JCSAT-2B 2016-05-06T0